## Using Local Ollama model
In past we used Open AI model which is paid one. You can try it with 5$ free tier. The limitations is number of token and API rate limit. 

Alternatively you can install Ollama, pull ollama model and use it locally. 

In this session, we will explore using Ollama model.

Before that, download and install ollama by following https://ollama.com/download

Ollama models: https://ollama.com/search 

- gpt-oss: https://ollama.com/library/gpt-oss 
- llama3.2: https://ollama.com/library/llama3.2 
- deepseek-r1: https://ollama.com/library/deepseek-r1

Pull below models by 

ollama pull [model name] or

ollama run [model name]

- llama3.2:latest
- gpt-oss:latest
- deepseek-r1:latest




#### Import libraries

In [ ]:
# To use local ollama model, we need to import AsyncOpenAI and OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, SQLiteSession
from dotenv import load_dotenv

#### Load openai api key from environment

In [ ]:
# Load environment variables
load_dotenv(override=True)

True

#### Define OpenAI client and llm model

In [ ]:
# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using client
model_name = "llama3.2:latest"
# model_name = "gpt-oss"

# gpt-oss is a thinking model. at least 13GB free RAM is required to run it locally. 
model = OpenAIChatCompletionsModel(model = model_name,openai_client= client)

#### Define Agent

In [ ]:
# Create an agent using the local model
local_agent = Agent(name="Bioinfo Local Agent",
                    instructions="You are a helpful bioinformatics assistant using local LLM server.",
                    model=model)

#### Run Agent as chat interface with session enabled

In [ ]:
session = SQLiteSession("bioinfo_session")

async def chat(message, history):
    result = await Runner.run(local_agent, message, session=session)
    return result.final_output


import gradio as gr
gr.ChatInterface(
    chat,
    title="Bioinformatics Expert Chatbot",
    description="Chat with the Bioinformatics Expert"
).launch()
